Copyright (c) 2025 Ken Barker

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Calculate Geodesic Inverse Accuracy Effect on Iterations

Calculate geodesics between positions at different levels of accuracy and show the number of iterations required.

This notebook compares calls of `aux_sphere_azimuth_length` with different tolerance values
to determine the effect of reducing accuracy on the number of iterations required to calculate a geodesic

Performs calculations and compares the results with data from Charles Karney's [Test data for geodesics](https://geographiclib.sourceforge.io/C++/doc/geodesic.html#testgeod).

The contents of the data file are as follows:

- 0-100000 entries randomly distributed
- 100000-150000 entries which are nearly antipodal
- 150000-200000 entries with short distances
- 200000-250000 entries with one end near a pole
- 250000-300000 entries with both ends near opposite poles
- 300000-350000 entries which are nearly meridional
- 350000-400000 entries which are nearly equatorial
- 400000-450000 entries running between vertices (α1 = α2 = 90°)
- 450000-500000 entries ending close to vertices

This file uses the first 100000 random entries.

In [ ]:
%matplotlib inline
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
from scipy import stats
from enum import Enum

from via_angle import Angle, Degrees, Radians
from via_sphere import LatLong, MIN_VALUE
from via_units import Metres
from via_ellipsoid import Ellipsoid, aux_sphere_azimuth_length, convert_radians_to_metres

## Read the GeodTest.dat file into a polars LazyFrame

In [ ]:
# The columns of the GeodTest.dat file
class Column(Enum):
    latitude_1   = 0
    longitude_1  = 1
    azimuth_1    = 2
    latitude_2   = 3
    longitude_2  = 4
    azimuth_2    = 5
    distance_m   = 6
    distance_deg = 7
    m12          = 8
    area         = 9

# Read the geodesic test data file into a polars LazyFrame: lf
# Select the random test data entries
size = 100000
pathname = 'https://sourceforge.net/projects/geographiclib/files/testdata/GeodTest.dat.gz/download'
lf = pl.scan_csv(pathname, separator=' ', has_header=False).select(
    ['column_1', 'column_2', 'column_3', 'column_4', 'column_5', 'column_7']
).head(size).collect()
lf.schema

### Calculate geodesic azimuths, distances and iterations for different tolerances

In [ ]:
%%time
delta_azimuth_default = np.empty(size)
delta_length_default = np.empty(size)
iterations_default = np.empty(size)

# Caculate the tolerance in Radians as the half the accuracy divided by the major axis length
tolerance_1mm = Radians(5e-4 / Ellipsoid.wgs84().a().v())
delta_azimuth_1mm = np.empty(size)
iterations_1mm = np.empty(size)
delta_length_1mm = np.empty(size)

tolerance_1m = Radians(0.5 / Ellipsoid.wgs84().a().v())
delta_azimuth_1m = np.empty(size)
iterations_1m = np.empty(size)
delta_length_1m = np.empty(size)

i = 0
for row in lf.rows():
    # Get departure and arrival positions from lf
    lat_1 = Angle(Degrees(row[Column.latitude_1.value]))
    lat_2 = Angle(Degrees(row[Column.latitude_2.value]))
    delta_long = Angle(Degrees(row[Column.longitude_2.value]))

    # calculate parametric latitudes from geodetic latitudes
    beta1 = Ellipsoid.wgs84().calculate_parametric_latitude(lat_1)
    beta2 = Ellipsoid.wgs84().calculate_parametric_latitude(lat_2)

    # solve the inverse geodesic problem on the auxiliary sphere with default tolerance
    azim_aux, length_aux, iters_aux = aux_sphere_azimuth_length(beta1, beta2, delta_long, Radians(MIN_VALUE), Ellipsoid.wgs84())
    delta_azimuth_default[i] = np.abs(azim_aux.to_degrees().v() - row[Column.azimuth_1.value])
    iterations_default[i] = iters_aux
    length_m = convert_radians_to_metres(beta1, azim_aux, length_aux, Ellipsoid.wgs84());
    delta_length_default[i] = np.abs(length_m.v() - row[5])

    # solve the inverse geodesic problem on the auxiliary sphere with 0.5 mm tolerance
    azim_aux, length_aux, iters_aux = aux_sphere_azimuth_length(beta1, beta2, delta_long, tolerance_1mm, Ellipsoid.wgs84())
    delta_azimuth_1mm[i] = np.abs(azim_aux.to_degrees().v() - row[Column.azimuth_1.value])
    iterations_1mm[i] = iters_aux
    length_m = convert_radians_to_metres(beta1, azim_aux, length_aux, Ellipsoid.wgs84());
    delta_length_1mm[i] = np.abs(length_m.v() - row[5])

    # solve the inverse geodesic problem on the auxiliary sphere with 0.5 m tolerance
    azim_aux, length_aux, iters_aux = aux_sphere_azimuth_length(beta1, beta2, delta_long, tolerance_1m, Ellipsoid.wgs84())
    delta_azimuth_1m[i] = np.abs(azim_aux.to_degrees().v() - row[Column.azimuth_1.value])
    iterations_1m[i] = iters_aux
    length_m = convert_radians_to_metres(beta1, azim_aux, length_aux, Ellipsoid.wgs84());
    delta_length_1m[i] = np.abs(length_m.v() - row[5])
    
    i += 1

In [ ]:
stats.describe(delta_azimuth_default)

In [ ]:
stats.describe(delta_azimuth_1mm)

In [ ]:
stats.describe(delta_azimuth_1m)

In [ ]:
stats.describe(delta_length_default)

In [ ]:
stats.describe(delta_length_1mm)

In [ ]:
stats.describe(delta_length_1m)

In [ ]:
stats.describe(iterations_default)

In [ ]:
stats.describe(iterations_1mm)

In [ ]:
stats.describe(iterations_1m)

In [ ]:
fig, axes = plt.subplots(3, 1, sharey=True)

axes[0].hist(iterations_default, bins=np.arange(6), align='left', color='#808080')
axes[0].set_ylabel('Default Tolerance')

axes[1].hist(iterations_1mm, bins=np.arange(6), align='left', color='#000080')
axes[1].set_ylabel('1mm Tolerance')

axes[2].hist(iterations_1m, bins=np.arange(6), align='left', color='#800000')
axes[2].set_xlabel('Iterations - Random Samples')
axes[2].set_ylabel('1m Tolerance')

# plt.savefig('../../docs/images/geodesic_inverse_accuracy_iterations.svg')

## Findings

Reducing the tolerance to 1mm, reduces the average number of iterations by nearly one
with a maximum length error of just over 1cm.

Further reducing the tolerance to 1m, barely reduced the average number of iterations
but the maximum length error increased to over 26m.

## Conclusion

A tolerance of 1mm may be suitable for navigation.